<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/final/single_cls_roberta_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate
!pip install torch tqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, concatenate_datasets
from datasets import ClassLabel, Value
from collections import Counter

sof_dataset_train = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/dataset/stackexchange_train_dataset_10000_title.csv',
    split='train'
)


# 결과를 확인
print(sof_dataset_train)

# 레이블의 데이터 건수 카운트
label_counter = Counter(example['label'] for example in sof_dataset_train)

print(label_counter)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
    num_rows: 10000
})
Counter({0: 2000, 1: 2000, 2: 2000, 3: 2000, 4: 2000})


In [3]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, concatenate_datasets
from datasets import ClassLabel, Value
from collections import Counter

sof_dataset_val = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/dataset/stackexchange_val_dataset_1000_title.csv',
    split='train'
)


# 결과를 확인
print(sof_dataset_val)

# 레이블의 데이터 건수 카운트
label_counter = Counter(example['label'] for example in sof_dataset_val)

print(label_counter)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
    num_rows: 1000
})
Counter({0: 200, 1: 200, 2: 200, 3: 200, 4: 200})


In [4]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, concatenate_datasets
from datasets import ClassLabel, Value
from collections import Counter

sof_dataset_test = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/dataset/stackexchange_test_dataset_1000_title.csv',
    split='train'
)


# 결과를 확인
print(sof_dataset_test)

# 레이블의 데이터 건수 카운트
label_counter = Counter(example['label'] for example in sof_dataset_test)

print(label_counter)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
    num_rows: 1000
})
Counter({0: 200, 1: 200, 2: 200, 3: 200, 4: 200})


# Tokenizing

In [6]:
from transformers import AutoTokenizer

sof_dataset= DatasetDict({
    "train": sof_dataset_train,
    "val" : sof_dataset_val,
    "test": sof_dataset_test
})

# DatasetDict 출력
print(sof_dataset)


tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

def tokenize_function(examples):
    return tokenizer(examples["title_content"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = sof_dataset.map(tokenize_function, batched=True)


DatasetDict({
    train: Dataset({
        features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
        num_rows: 10000
    })
    val: Dataset({
        features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['title_content', 'label', 'Tags_new', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS'],
        num_rows: 1000
    })
})


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

# Model / LoRA config

In [7]:
from peft import LoraConfig, TaskType
from peft import get_peft_model
from transformers import RobertaForSequenceClassification

# lora_config = LoraConfig(
#     task_type=TaskType.SEQ_CLS, r=8,lora_alpha=16, lora_dropout=0.1
# )

model = RobertaForSequenceClassification.from_pretrained(
    'FacebookAI/roberta-large',
    num_labels=5
)

# model = get_peft_model(model, lora_config)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Fine-tuning

In [9]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metrics = {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "macro_precision": precision_metric.compute(predictions=predictions, references=labels, average="macro")["precision"],
        "macro_recall": recall_metric.compute(predictions=predictions, references=labels, average="macro")["recall"],
        "macro_f1": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"],
        "micro_precision": precision_metric.compute(predictions=predictions, references=labels, average="micro")["precision"],
        "micro_recall": recall_metric.compute(predictions=predictions, references=labels, average="micro")["recall"],
        "micro_f1": f1_metric.compute(predictions=predictions, references=labels, average="micro")["f1"],
        "weighted_precision": precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "weighted_recall": recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"],
        "weighted_f1": f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"],
    }
    return metrics

# training_args = TrainingArguments(output_dir="/content/drive/MyDrive/LoRA/CLASSIFICATION1_roberta_LoRA_stackexchange",
#                                   evaluation_strategy="epoch",
#                                   num_train_epochs=10)


# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset= tokenized_datasets["train"],
#     eval_dataset = tokenized_datasets["test"],
#     compute_metrics=compute_metrics,
# )

# trainer.train()


# Trainer 객체 생성
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 모델 평가
results = trainer.evaluate(tokenized_datasets["test"])
print(results)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


{'eval_loss': 1.6388911008834839, 'eval_model_preparation_time': 0.0057, 'eval_accuracy': 0.2, 'eval_macro_precision': 0.04, 'eval_macro_recall': 0.2, 'eval_macro_f1': 0.06666666666666667, 'eval_micro_precision': 0.2, 'eval_micro_recall': 0.2, 'eval_micro_f1': 0.2, 'eval_weighted_precision': 0.04, 'eval_weighted_recall': 0.2, 'eval_weighted_f1': 0.06666666666666665, 'eval_runtime': 20.8345, 'eval_samples_per_second': 47.997, 'eval_steps_per_second': 6.0}


# Inference

In [10]:
from transformers import pipeline

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

label_map = {
    0: "biology",
    1: "cooking",
    2: "diy",
    3: "travel",
    4: "stackoverflow"
}

# 주어진 텍스트를 토대로 예측을 5번 실행
texts = [
    "What are the theoretical and actual (measured) minimum water potentials in plants?",
    "What tastes like marigold? <p>I'm  looking to make Georgian Satsivi ",
    "How can I intentionally make my toilet make this noise? <p>For Halloween I want to haunt my toilets and have them make the noise heard here:",
    "What are the hours of operation of the Ankara Metro? <p>What are the times of the first and last trains on the Ankara Metro?</p>",
    "How to use \"HTML form target self\" ? I am new to HTML and need to complete a simple task.",
]

# 각 텍스트에 대해 예측 실행 및 레이블 출력
for i, text in enumerate(texts):
    prediction = classifier(text)
    predicted_label = prediction[0]['label']
    predicted_score = prediction[0]['score']

    # 레이블 이름을 가져옵니다 (여기서는 RoBERTa 모델의 레이블이 "LABEL_0", "LABEL_1" 등으로 되어있을 수 있으므로 매핑이 필요합니다).
    label_name = label_map[int(predicted_label.split('_')[-1])]
    print(f"Text {i+1}: Predicted label: {label_name}, Score: {predicted_score:.4f}")

Text 1: Predicted label: diy, Score: 0.3052
Text 2: Predicted label: diy, Score: 0.3008
Text 3: Predicted label: diy, Score: 0.2989
Text 4: Predicted label: diy, Score: 0.3053
Text 5: Predicted label: diy, Score: 0.2909
